In [17]:
import json
import nltk
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from nltk.corpus import wordnet
import string

In [18]:
#load the corpus as a string

source_text = ""

substring1 = "buy now"
substring2 = "please retweet my store"
substring3 = "giveaway"
substring4 = "amazon.com"
substring5 = "preorder"

with open("tweets.csv", "r") as f:
    for line in f:
        if substring1 in line or substring2 in line or substring3 in line or substring4 in line or substring5 in line:
            continue
        else:
            source_text += line
            source_text += " "

In [19]:
#split string into sentences

tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
punctuations = list(string.punctuation)

# raw_sentences = tokenizer.tokenize(source_text)
raw_sentences = [i for i in tokenizer.tokenize(source_text) if i not in punctuations]

In [20]:
raw_sentences[900]

'🌅 Click our the link to join our newsl ...  http://bit.ly/poetrysubscribe\xa0 pic.twitter.com/QP6k7SzSVB"\n Amazon looks to airports for cashier-less stores: report  https://www.fastcompany.com/90278124/amazon-looks-to-airports-for-cashier-less-stores-report\xa0… #Amazon #retail\n "OK, so I just watched Coldplay Head Full of Dreams on Amazon Prime.'

In [21]:
#clean each sentence, saving semantically meaningful words into an array

substring1 = "http"
substring2 = ".com"

def sentence_to_wordlist(raw):
    stopWords = set(stopwords.words('english'))
    words = raw.lower().split()
    tokenized = tokenizer.tokenize(raw)
    sentenceArray = []

    for word in words:
        if word in stopWords or substring1 in word or substring2 in word:
            continue
        else:
            sentenceArray.append(word)
            
    # Create a list of words
    return sentenceArray

In [22]:
cleaned_sentences = []
for raw in raw_sentences:
    cleaned_sentences.append(sentence_to_wordlist(raw))

In [23]:
print(cleaned_sentences[900])

['🌅', 'click', 'link', 'join', 'newsl', '...', 'amazon', 'looks', 'airports', 'cashier-less', 'stores:', 'report', '…', '#amazon', '#retail', '"ok,', 'watched', 'coldplay', 'head', 'full', 'dreams', 'amazon', 'prime.']


In [24]:
#build the w2v model

model = w2v.Word2Vec(sg=1,seed=1,workers= 4,size=100,min_count=3,window=15,sample=1e-3)
model.build_vocab(cleaned_sentences)
model.train(cleaned_sentences, total_examples=model.corpus_count,epochs=model.iter)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


(831617, 1098595)

In [25]:
print(model)

Word2Vec(vocab=9975, size=100, alpha=0.025)


In [26]:
#save the model as a lookup dictionary json

vectorsDict = {}
for sentence in cleaned_sentences:
    for word in sentence:
        keys = list(vectorsDict.keys())
        if word not in keys:
            try:
                vectorsDict[word] = model.wv[word].tolist()
            except:
                continue

In [28]:
with open("AmazonTweets.json", "w+") as f:
    json.dump(vectorsDict, f)